<a href="https://colab.research.google.com/github/EVA4-RS-Group/Phase2/blob/master/S6_GAN/EVA4_P2_S6_GenerativeAdversarialNetwork_Data_Preprocessing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
import glob

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [135]:
input_dir = "/content/drive/My Drive/EVA4/Phase2/S6/processed_images_step1/"
save_dir = "/content/drive/My Drive/EVA4/Phase2/S6/processed_images_step2/"
raw_file_list = [file_path for file_path in glob.iglob(f"{input_dir}*.*",recursive=True)]
global img_idx, img_cur, img_mod
img_idx = 0

In [136]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
from IPython.display import clear_output
desired_size = 100


def get_image():
  global img_idx, img_cur, img_mod
  if img_idx>=0 and img_idx<len(raw_file_list):
    pass
  elif img_idx<0:
    img_idx = len(raw_file_list)-1
  else:
    img_idx = 0

  img_cur = Image.open(raw_file_list[img_idx]).convert("RGBA")
  img_mod = None
  
  return 0

out = widgets.Output()
words = ['Next', 'Previous', 'Flip', 'Resize', 'Save']
items = [Button(description=w) for w in words]
combined_buttons = HBox(items)

def plot_image_next(_,inc=1):

  with out:
    global img_idx, img_cur
    _ = get_image()
    clear_output()
    plt.figure(figsize=(6,6))
    plt.imshow(img_cur)
    img_idx += inc
    plt.title(raw_file_list[img_idx].split("/")[-1])
    plt.show()

items[0].on_click(plot_image_next)

def plot_image_prev(_):
  return plot_image_next(_,inc=-1)

items[1].on_click(plot_image_prev)

def plot_image_flip(_):

  with out:
    global img_idx, img_cur, img_mod
    if img_mod is None:
      img_mod = ImageOps.mirror(img_cur)
    else:
      img_mod = ImageOps.mirror(img_mod)
    clear_output()
    plt.figure(figsize=(6,6))
    plt.imshow(img_mod)
    plt.title(raw_file_list[img_idx].split("/")[-1])
    plt.show()

items[2].on_click(plot_image_flip)

def plot_cropped_image(_):
  with out:
    global img_idx, img_cur, img_mod
    if not bool(img_mod):
      img_mod = img_cur
    
    imageSize = img_mod.size
    imageBox = img_mod.getbbox()

    imageComponents = img_mod.split()

    rgbImage = Image.new("RGB", imageSize, (0,0,0))
    rgbImage.paste(img_mod, mask=imageComponents[3])
    croppedBox = rgbImage.getbbox()
    if imageBox != croppedBox:
        img_mod=img_mod.crop(croppedBox)


    old_size = img_mod.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = img_mod.resize(new_size)
    # create a new image and paste the resized on it

    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    img_mod = ImageOps.expand(im, padding)

    clear_output()
    plt.figure(figsize=(4,4))
    plt.imshow(img_mod)
    plt.title(raw_file_list[img_idx].split("/")[-1])
    plt.show()

items[3].on_click(plot_cropped_image)

def image_save(_):
  global img_idx, img_cur, img_mod
  if not bool(img_mod):
    img_mod = img_cur
  
  png = img_mod
  png.load() # required for png.split()

  img_new = Image.new("RGB", png.size, (255, 255, 255))
  img_new.paste(png, mask=png.split()[3]) # 3 is the alpha channel

  img_new.save(save_dir+"img_{0:03d}.jpg".format(img_idx))

items[4].on_click(image_save)

VBox([combined_buttons,out])